# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [14]:
#r "nuget: Microsoft.SemanticKernel, 1.16.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.16.0-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.16.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.16.0-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [15]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

Also import a plugin class
```csharp
public class StaticTextPlugin
{
    [KernelFunction, Description("Change all string chars to uppercase")]
    public static string Uppercase([Description("Text to uppercase")] string input) =>
        input.ToUpperInvariant();

    [KernelFunction, Description("Append the day variable")]
    public static string AppendDay(
        [Description("Text to append to")] string input,
        [Description("Value of the day to append")] string day) =>
        input + day;
}
```


In [16]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "plugins");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

#!import plugins/StaticTextPlugin.cs
kernel.ImportPluginFromType<StaticTextPlugin>();

Define your ASK. What do you want the Kernel to do?

In [17]:
#pragma warning disable SKEXP0060

var ask = "Gumawa ka ng tula tungkol sa GinPom na inumin, tapos isalin mo sa Ingles.";
var plan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine(plan);

{{!-- Step 0: Extract Key Values --}}
{{set "drinkName" "GinPom"}}
{{set "englishText" ""}}

{{!-- Step 1: Create a poem about the GinPom drink --}}
{{set "poem" (WriterPlugin-ShortPoem input=("Tungkol sa " drinkName " na inumin, Masarap pag-inuman sa hapon."))}}

{{!-- Step 2: Translate the poem to English --}}
{{set "englishText" (WriterPlugin-Translate input=poem language="English")}}

{{json englishText}}


### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [19]:
#pragma warning disable SKEXP0060

var result = await plan.InvokeAsync(kernel);
result


Error: HandlebarsDotNet.HandlebarsCompilerException: Found a sub-expression that does not contain a path expression
   at HandlebarsDotNet.Compiler.SubExpressionConverter.BuildSubExpression(IEnumerator`1 enumerator)
   at HandlebarsDotNet.Compiler.SubExpressionConverter.AccumulateSubExpression(IEnumerator`1 enumerator)
   at HandlebarsDotNet.Compiler.SubExpressionConverter.BuildSubExpression(IEnumerator`1 enumerator)
   at HandlebarsDotNet.Compiler.SubExpressionConverter.ConvertTokens(IEnumerable`1 sequence)+MoveNext()
   at System.Collections.Generic.List`1..ctor(IEnumerable`1 collection)
   at System.Linq.Enumerable.ToList[TSource](IEnumerable`1 source)
   at HandlebarsDotNet.Compiler.SubExpressionConverter.Convert(IEnumerable`1 sequence)
   at HandlebarsDotNet.Compiler.ExpressionBuilder.ConvertTokensToExpressions(IEnumerable`1 tokens, ICompiledHandlebarsConfiguration configuration)
   at HandlebarsDotNet.Compiler.HandlebarsCompiler.Compile(ExtendedStringReader source, CompilationCon